In [76]:
import pandas as pd

**Import Data**

In [77]:
census = pd.read_csv('../data/census_bureau_acs_county_2018_5yr.csv')
covid = pd.read_csv('../data/covid19_nyt_us_counties.csv')
hospital = pd.read_csv('../data/us_healthcare_capacity-county-CovidCareMap.csv')

**Covid Data**

In [78]:
# interested in county level data
# grab max as data is already additive
covid = covid.groupby('county_fips_code').max()

In [79]:
covid.isna().sum() / covid.shape[0]

date               0.000000
county             0.000000
state_name         0.000000
confirmed_cases    0.000000
deaths             0.024239
dtype: float64

In [80]:
# check null values of deaths for pattern
covid[covid['deaths'].isna()]['state_name'].value_counts()

Puerto Rico    78
Name: state_name, dtype: int64

In [81]:
# all data is related to Peurto Rico and less than 3%
covid.dropna(inplace=True)

In [82]:
# create target
covid['death_rate'] = covid['deaths'] / covid['confirmed_cases']
covid = covid[['death_rate']]
covid.describe()

,death_rate
count,3140.000000
mean,0.019864
std,0.010036
min,0.000000
25%,0.013215
50%,0.018458
75%,0.024953
max,0.097561


**Hospital Data**

In [83]:
hospital.isna().sum() / hospital.shape[0]

fips_code                                     0.000000
State                                         0.000000
County Name                                   0.000000
Staffed All Beds                              0.000000
Staffed ICU Beds                              0.000000
Licensed All Beds                             0.000000
All Bed Occupancy Rate                        0.030351
ICU Bed Occupancy Rate                        0.444225
Population                                    0.000000
Population (20+)                              0.000000
Population (65+)                              0.000000
Staffed All Beds [Per 1000 People]            0.011037
Staffed All Beds [Per 1000 Adults (20+)]      0.011037
Staffed All Beds [Per 1000 Elderly (65+)]     0.011037
Staffed ICU Beds [Per 1000 People]            0.011037
Staffed ICU Beds [Per 1000 Adults (20+)]      0.011037
Staffed ICU Beds [Per 1000 Elderly (65+)]     0.011037
Licensed All Beds [Per 1000 People]           0.011037
Licensed A

In [84]:
# drop null columns then rows
hospital.drop(columns=['ICU Bed Occupancy Rate', 
                       'ICU Bed Source Last Updated'], inplace=True)
hospital.dropna(inplace=True)

# set index in in preparation to merge with covid data
hospital.set_index('fips_code', inplace=True)

In [85]:
hospital.columns

Index(['State', 'County Name', 'Staffed All Beds', 'Staffed ICU Beds',
       'Licensed All Beds', 'All Bed Occupancy Rate', 'Population',
       'Population (20+)', 'Population (65+)',
       'Staffed All Beds [Per 1000 People]',
       'Staffed All Beds [Per 1000 Adults (20+)]',
       'Staffed All Beds [Per 1000 Elderly (65+)]',
       'Staffed ICU Beds [Per 1000 People]',
       'Staffed ICU Beds [Per 1000 Adults (20+)]',
       'Staffed ICU Beds [Per 1000 Elderly (65+)]',
       'Licensed All Beds [Per 1000 People]',
       'Licensed All Beds [Per 1000 Adults (20+)]',
       'Licensed All Beds [Per 1000 Elderly (65+)]', 'ICU Bed Source'],
      dtype='object')

In [86]:
hospital = hospital[['Staffed All Beds',
                     'Staffed ICU Beds',
                     'Licensed All Beds',
                     'All Bed Occupancy Rate',
                     'Population',
                     'Population (20+)',
                     'Population (65+)',
                     'Staffed All Beds [Per 1000 People]',
                     'Staffed All Beds [Per 1000 Adults (20+)]',
                     'Staffed All Beds [Per 1000 Elderly (65+)]',
                     'Staffed ICU Beds [Per 1000 People]',
                     'Staffed ICU Beds [Per 1000 Adults (20+)]',
                     'Staffed ICU Beds [Per 1000 Elderly (65+)]',
                     'Licensed All Beds [Per 1000 People]',
                     'Licensed All Beds [Per 1000 Adults (20+)]',
                     'Licensed All Beds [Per 1000 Elderly (65+)]']]
hospital.describe()

,Staffed All Beds,Staffed ICU Beds,Licensed All Beds,All Bed Occupancy Rate,Population,Population (20+),Population (65+),Staffed All Beds [Per 1000 People],Staffed All Beds [Per 1000 Adults (20+)],Staffed All Beds [Per 1000 Elderly (65+)],Staffed ICU Beds [Per 1000 People],Staffed ICU Beds [Per 1000 Adults (20+)],Staffed ICU Beds [Per 1000 Elderly (65+)],Licensed All Beds [Per 1000 People],Licensed All Beds [Per 1000 Adults (20+)],Licensed All Beds [Per 1000 Elderly (65+)]
count,2436.000000,2436.000000,2436.000000,2436.000000,2.436000e+03,2.436000e+03,2.436000e+03,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000,2436.000000
mean,332.106322,32.389163,428.863711,0.416203,1.296366e+05,9.711367e+04,2.064483e+04,2.665162,3.547758,14.597846,0.164152,0.218610,0.961966,3.453977,4.594261,18.994130
std,1003.550685,103.038696,1279.834426,0.209624,3.747552e+05,2.809996e+05,5.350184e+04,2.417202,3.175213,12.894066,0.231258,0.307273,1.375051,3.906481,5.076848,20.905135
min,2.000000,0.000000,0.000000,0.000000,1.087000e+03,8.800000e+02,2.540000e+02,0.092000,0.128000,0.493000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,0.000000,25.000000,0.240000,1.550450e+04,1.171075e+04,3.064000e+03,1.291000,1.710500,6.940500,0.000000,0.000000,0.000000,1.588000,2.113750,8.511000
50%,50.000000,6.000000,76.000000,0.400000,3.563350e+04,2.688050e+04,6.643500e+03,2.038500,2.720500,11.203000,0.119000,0.160000,0.658000,2.626500,3.506000,14.336000
75%,209.000000,20.000000,272.250000,0.590000,9.681875e+04,7.229200e+04,1.637425e+04,3.138000,4.210000,17.792750,0.242250,0.322250,1.354500,4.081500,5.441500,22.717250
max,23071.000000,2305.000000,27400.000000,1.000000,1.010552e+07,7.658126e+06,1.375957e+06,30.373000,39.077000,155.666000,3.728000,4.797000,18.824000,97.226000,120.423000,463.326000


**Census Data**

In [87]:
len(census.columns[census.isna().sum() > 0])

121

In [88]:
census[census.columns[census.isna().sum() > 0]].isna().sum().describe()

count     121.000000
mean      274.206612
std       888.196768
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max      3220.000000
dtype: float64

In [89]:
one_null_lst = ['geo_id'] + list(census.columns[census.isna().sum() == 1])
census[one_null_lst][census[one_null_lst].isna().any(axis=1)]

,geo_id,pop_16_over,pop_25_64,median_income,income_per_capita,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,...,employed_science_management_admin_waste,employed_transportation_warehousing_utilities,employed_wholesale_trade,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed
0,35039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
census.shape

(3220, 242)

In [91]:
census[list(census.columns[(census.isna().sum() > 5) & (census.isna().sum() < 3220)])].isna().sum()

population_1_year_and_over                  79
not_us_citizen_pop                          78
different_house_year_ago_different_city     79
different_house_year_ago_same_city          79
aggregate_travel_time_to_work              158
less_than_high_school_graduate              79
high_school_including_ged                   79
bachelors_degree_2                          79
graduate_professional_degree                79
some_college_and_associates_degree          79
dtype: int64

In [92]:
census[census.columns[census.isna().sum() == 1]].dropna().shape

(3219, 95)

In [93]:
census.shape

(3220, 242)

In [94]:
census = census.drop(list(census.columns[census.isna().sum() > 100]), axis=1)
census = census.dropna()

In [95]:
# poverty rate == poverty / pop_determined_poverty_status
census['poverty_rate'] = census['poverty'] / census['pop_determined_poverty_status']

In [96]:
census = census.set_index('geo_id')

**Merge Datasets**
* covid
* census
* hospital

In [97]:
df = census.merge(covid, left_index=True, right_index=True)
df = df.merge(hospital, left_index=True, right_index=True)

In [98]:
df['pct_nonfamily_house'] = df['nonfamily_households'] / df['households']
df['pct_family_house'] = df['family_households'] / df['households']
df['pct_male_male_households'] = df['male_male_households'] / df['households']
df['pct_female_female_households'] = df['female_female_households'] / df['households']
df['pct_male'] = df['male_pop'] / df['total_pop']
df['pct_female'] = df['female_pop'] / df['total_pop']
df['pct_white'] = df['white_pop'] / df['total_pop']
df['pct_black'] = df['black_pop'] / df['total_pop']
df['pct_asian'] = df['asian_pop'] / df['total_pop']
df['pct_hispanic'] = df['hispanic_pop'] / df['total_pop']
df['pct_amerindian'] = df['amerindian_pop'] / df['total_pop']
df['pct_other_race'] = df['other_race_pop'] / df['total_pop']
df['pct_two_or_more_races'] = df['two_or_more_races_pop'] / df['total_pop']
df['pct_not_hispanic'] = df['not_hispanic_pop'] / df['total_pop']

In [99]:
df.shape

(2430, 262)

**Save Dataset**

In [100]:
df.to_csv('../data/sam_dataset.csv')